# Objective

Given a review , we have to determine whether the review is positive (4 or 5) or negative(1 or 2) by applying different models of machine learning and we can see the best result out of different classification models to classify the reviews.

# Dataset

The dataset is available on the kaggle https://www.kaggle.com/snap/amazon-fine-food-reviews/home. This dataset consists of 568,454 reviews of fine food from Amazon.This dataset includes the reviews from Oct 1999 to Oct 2012.The dataset is avaliable in two forms

1.csv file <br />
2.SQLite Databas

# Importing the Required Libraries 

In [1]:
import numpy as np 
import pandas as pd
# import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Importing the Data

In [2]:
pwd

'C:\\Users\\Vivek'

In [3]:
cd E:\Amazon Fine Food Reviews

E:\Amazon Fine Food Reviews


In [4]:
pwd

'E:\\Amazon Fine Food Reviews'

In [5]:
df = pd.read_csv('Reviews.csv')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
df.shape

(568454, 10)

### Creating New Feature named as Sentiment
Score = 3 Sentiment = 0 (Neutral) <br/>
Score > 3 Sentiment = 1 (Positive) <br/>
Score < 3 Sentiment = -1 (Negative) <br/>

In [7]:
df['Sentiment'] = df["Score"].apply(lambda x: 1 if x > 3 else (0 if x == 3 else -1))

In [8]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,-1
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,-1
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1


# Data Cleaning 

This stage is very important and quite time taking stage. You have to understand own your own what cleaning , you should do in our dataset and what type of cleaning is required according to our problem.Here, you will observe the duplicates in this dataset and you need to remove those duplicates in order to get the unbiased result for the analysis of data.One more important thing we need to focus on is that this dataset is based on timestamp so we need to sort our data according to the timestamp so that our model could perform very well on future unseen data (new reviews), for example, to deploy sentiment analysis of reviews in the system , each day there will be new reviews added for the different products which will act as new data for the system’s model so in order to perform well on new data with same results as it is preforming on old data, you need to sort them acc. to the timestamp.

Helpfulness - It is the ratio of people find this review helpful to total reaction on this review 
<br/>
So in this Numerator should always be less than or equals to Denominator. But we found some review where Numerator is greater than Denominator.

In [9]:
df[df.HelpfulnessNumerator > df.HelpfulnessDenominator]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
44736,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...,1
64421,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...,1


In [10]:
df = df[df.HelpfulnessNumerator <= df.HelpfulnessDenominator]

In [11]:
df[df['UserId'] == 'A395BORC6FGVXV']

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,-1
136303,136304,B002Y7526Y,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,-1
544172,544173,B000U9WZ54,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,-1


In [12]:
filtered_df = df.drop_duplicates(subset = {'UserId','ProfileName','Time'} ,keep = 'first', inplace = False)

In [13]:
final = filtered_df.sort_values('Time',axis= 0,inplace = False , na_position = 'last',ascending = True)

In [14]:
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
150523,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,5,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,1
150500,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,5,940809600,This whole series is great way to spend time w...,I can remember seeing the show when it aired o...,1
451855,451856,B00004CXX9,AIUWLEQ1ADEG5,Elizabeth Medina,0,0,5,944092800,Entertainingl Funny!,Beetlejuice is a well written movie ..... ever...,1
230284,230285,B00004RYGX,A344SMIA5JECGM,Vincent P. Ross,1,2,5,944438400,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta...",1
451854,451855,B00004CXX9,AJH6LUC1UT1ON,The Phantom of the Opera,0,0,5,946857600,FANTASTIC!,Beetlejuice is an excellent and funny movie. K...,1


In [15]:
final.shape

(352784, 11)

In [16]:
print ("Data remained after data cleaning is {} %.".format((final.shape[0]/df.shape[0])*100))

Data remained after data cleaning is 62.06047300387719 %.


In [17]:
final['Sentiment'].value_counts()

 1    273904
-1     52692
 0     26188
Name: Sentiment, dtype: int64

# Text Preprocessing

Data preprocessing is also an important process in which you transform our data before applying the different models on dataset so that you can achieve the better results. In this stage , you can perform certain steps on our reviews(text) as mentioned below:-

* Begin with defining your own function which will remove the html tags or we can use Beautiful Soup .
* Removal of punctuations or limited set of special characters like . or “,” or removing whitespaces or etc from the reviews.
* Check if the word is made up of english letters and is not alpha-numeric.
* Check if the length of word is greater than 2 (as there is no adjective in 2 letters).
* uppercase to lower case conversion.
* Removal of Stopwords(commonly used word such as “a”, “the” etc to be ignored).
* Stemming(Snowball Stemming) the word of each reviews.

In [18]:
def cleanhtml(sent):# function for cleaning html tags
    cleanr = re.compile('<.*?>')
    cleaned = re.sub(cleanr,' ',sent)
    return cleaned

In [19]:
def cleanpunc(sent):# function for cleaning punctuations
    clean = re.sub(r'[?|!|$|#|\'|"|:]',r'',sent)
    clean = re.sub(r'[,|(|)|.|\|/]',r' ',clean)
    return clean

In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vivek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
stop = set(stopwords.words("english"))


In [22]:
from nltk.stem.snowball import SnowballStemmer
st = SnowballStemmer("english")
st.stem('burned')

'burn'

In [24]:
i=0
all_positive_reviews = []
all_negative_reviews = []
all_neutral_reviews = []

final_string = []
stem_data = " "
for p in final['Text'].values:
    filtered_sens = []#filtered word
    p = cleanhtml(p)
    for w in p.split():
       # print(w)
        punc = cleanpunc(w)
        for s in punc.split():
            #print(w)
            if (s.isalpha()) & (len(s)>2):
                if s.lower() not in stop:
                    stem_data = (st.stem(s.lower())).encode('utf8')
                    filtered_sens.append(stem_data)
                    if (final['Sentiment'].values)[i] == 1:
                        all_positive_reviews.append(stem_data)
                    if (final['Sentiment'].values)[i] == -1:
                        all_negative_reviews.append(stem_data)
                    if (final['Sentiment'].values)[i] == 0:
                        all_negative_reviews.append(stem_data)    
                else:
                    continue
            else:
                continue
    #print(filtered_sens)
    str1 = b" ".join(filtered_sens)
    #print(str1)
    final_string.append(str1)
    i+=1

In [25]:
final['CleanedText'] = final_string
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment,CleanedText
150523,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,5,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,1,b'witti littl book make son laugh loud recit c...
150500,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,5,940809600,This whole series is great way to spend time w...,I can remember seeing the show when it aired o...,1,b'rememb see show air televis year ago child s...
451855,451856,B00004CXX9,AIUWLEQ1ADEG5,Elizabeth Medina,0,0,5,944092800,Entertainingl Funny!,Beetlejuice is a well written movie ..... ever...,1,b'beetlejuic well written movi everyth excel a...
230284,230285,B00004RYGX,A344SMIA5JECGM,Vincent P. Ross,1,2,5,944438400,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta...",1,b'twist rumplestiskin captur film star michael...
451854,451855,B00004CXX9,AJH6LUC1UT1ON,The Phantom of the Opera,0,0,5,946857600,FANTASTIC!,Beetlejuice is an excellent and funny movie. K...,1,b'beetlejuic excel funni movi keaton hilari wa...


In [48]:
# final.to_csv('Reviews_Stamming.csv')

In [26]:
final = final.sort_values('Time',axis= 0,inplace = False , na_position = 'last',ascending = True)
X = final['CleanedText'].values
X = X[:100000]
y = final['Score'].values
y = y[:100000]

In [27]:
X_train ,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,stratify = y)

# Logistic regression on bow

In [28]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [29]:
count_vect = CountVectorizer() #in scikit-learn
bow_train = count_vect.fit_transform(X_train)
bow_test = count_vect.transform(X_test)
#count_vect.get_feature_names()
bow_train.shape

(70000, 32115)

In [37]:
tunned_param = [{'C' : [10** -4, 10**-3,10**-2,10**-1,1,10,100,1000,10000]}]
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
model  = GridSearchCV(LogisticRegression(class_weight = 'balanced', max_iter=1000),tunned_param , scoring = 'accuracy',cv = 10,n_jobs = -1)
model.fit(bow_train,y_train)
print(model.best_estimator_)
pred = model.predict(bow_test)
print('Accuracy ',accuracy_score(y_test,pred)*100)
LogisticRegression()

LogisticRegression(C=0.01, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Accuracy  61.693333333333335


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# On 3 class

In [39]:
final = final.sort_values('Time',axis= 0,inplace = False , na_position = 'last',ascending = True)
X = final['CleanedText'].values
X = X[:100000]
y = final['Sentiment'].values
y = y[:100000]

In [40]:
X_train ,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,stratify = y)

In [41]:
count_vect = CountVectorizer() #in scikit-learn
bow_train = count_vect.fit_transform(X_train)
bow_test = count_vect.transform(X_test)
#count_vect.get_feature_names()
bow_train.shape

(70000, 32026)

In [42]:
tunned_param = [{'C' : [10** -4, 10**-3,10**-2,10**-1,1,10,100,1000,10000]}]
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
model  = GridSearchCV(LogisticRegression(class_weight = 'balanced', max_iter=1000),tunned_param , scoring = 'accuracy',cv = 10,n_jobs = -1)
model.fit(bow_train,y_train)
print(model.best_estimator_)
pred = model.predict(bow_test)
print('Accuracy ',accuracy_score(y_test,pred)*100)

LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Accuracy  78.53999999999999


In [5]:
final = pd.read_csv("Reviews_Stamming.csv")
final.head()

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment,CleanedText
0,150523,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,5,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,1,b'witti littl book make son laugh loud recit c...
1,150500,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,5,940809600,This whole series is great way to spend time w...,I can remember seeing the show when it aired o...,1,b'rememb see show air televis year ago child s...
2,451855,451856,B00004CXX9,AIUWLEQ1ADEG5,Elizabeth Medina,0,0,5,944092800,Entertainingl Funny!,Beetlejuice is a well written movie ..... ever...,1,b'beetlejuic well written movi everyth excel a...
3,230284,230285,B00004RYGX,A344SMIA5JECGM,Vincent P. Ross,1,2,5,944438400,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta...",1,b'twist rumplestiskin captur film star michael...
4,451854,451855,B00004CXX9,AJH6LUC1UT1ON,The Phantom of the Opera,0,0,5,946857600,FANTASTIC!,Beetlejuice is an excellent and funny movie. K...,1,b'beetlejuic excel funni movi keaton hilari wa...


In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
final = final.sort_values('Time',axis= 0,inplace = False , na_position = 'last',ascending = True)
X = final['CleanedText'].values

y = final['Sentiment'].values


In [8]:
X_train ,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,stratify = y)

# Logistic regression on tfidf

In [9]:
tfidf_vect = TfidfVectorizer()
tfidf_train = tfidf_vect.fit_transform (X_train)
tfidf_test = tfidf_vect.transform(X_test)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean = False)
X_tr = sc.fit_transform(tfidf_train)
X_te = sc.transform(tfidf_test)
param = [{'C':[10**i for i in range(-3,4)]}]
gd = GridSearchCV(LogisticRegression(class_weight = 'balanced', max_iter=1000),param,cv = 10, scoring = 'accuracy',n_jobs = -1)
gd.fit(X_tr,y_train)
print(gd.best_estimator_)
pred = gd.predict(X_te)
print('Accuracy is ',accuracy_score(y_test,pred)*100)

LogisticRegression(C=0.001, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Accuracy is  75.13511470577119
